In [6]:
%pip install langchain openai faiss-cpu tiktoken pypdf langchain_community

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import os
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

# Load medical documents
docs_path = "../docs"
documents = []

for file in os.listdir(docs_path):
    file_path = os.path.join(docs_path, file)

    # Load PDFs with PyPDFLoader
    if file.endswith(".pdf"):
        loader = PyPDFLoader(file_path)
        documents.extend(loader.load())

    # Load text files with TextLoader
    elif file.endswith(".txt"):
        loader = TextLoader(file_path)
        documents.extend(loader.load())

# Split into smaller chunks for better retrieval
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

print(f"Loaded {len(chunks)} document chunks successfully!")


Loaded 5 document chunks successfully!


In [8]:
%pip install -q sentence-transformers pymilvus


^C
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from pymilvus import MilvusClient, DataType

CLUSTER_ENDPOINT = "https://in03-22706440ab83439.serverless.gcp-us-west1.cloud.zilliz.com"
TOKEN = "fd3df03a845da1e42df2a2975f1d4733f62b044415d021f9f6047ef1f631504509c1a870e1370ebc8587c04883fae0c72c886204"

# 1. Set up a Milvus client
client = MilvusClient(
    uri=CLUSTER_ENDPOINT,
    token=TOKEN 
)

In [13]:
# 2. Create a collection in quick setup mode
client.create_collection(
    collection_name="documents",
    dimension=5 # The dimensionality should be an integer greater than 1.
)

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Milvus
from pymilvus import Collection, FieldSchema, CollectionSchema, DataType, connections, utility

# Define the collection name
collection_name = "document_embeddings"

# Connect to Milvus (ensure you've set up the connection correctly)
connections.connect("default", uri=CLUSTER_ENDPOINT, token=TOKEN)

# # List all collections in the connected Milvus instance
# existing_collections = utility.list_collections()

# # Check if the collection exists or create it
# if collection_name not in existing_collections:
#     # Define the schema and create the collection
#     embedding_dim = 768  # Example dimension, make sure this matches your model

#     fields = [
#         FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=embedding_dim),
#         FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
#     ]
#     schema = CollectionSchema(fields, description="Document embeddings for retrieval")

#     # Create the collection
#     collection = Collection(name=collection_name, schema=schema)
# else:
#     collection = Collection(collection_name)

# print(f"Collection '{collection_name}' is ready.")


Collection 'document_embeddings' is ready.


In [ ]:
from pymilvus import connections

# Disconnect from all existing connections
connections.disconnect("default")

# Now try to connect again
connections.connect("default", uri=CLUSTER_ENDPOINT, token=TOKEN)
print("Successfully connected to Milvus!")


Successfully connected to Milvus!


In [ ]:
# from pymilvus import connections, utility

# connections.connect(alias="default", uri=CLUSTER_ENDPOINT, token=TOKEN)
# if utility.has_collection("document_embeddings"):
#     utility.drop_collection("document_embeddings")
#     print("🗑️ Dropped old collection.")


🗑️ Dropped old collection.


In [28]:
# Load your local embeddings model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Store the document embeddings into Milvus
try:
    vector_store = Milvus.from_documents(
        chunks,
        embeddings,
        collection_name=collection_name,
        connection_args={
            "uri": CLUSTER_ENDPOINT,
            "token": TOKEN,  # omit if unused
        }
    )
except Exception as e:
    print(f"❌ Failed to connect to Milvus cluster: {e}")

In [29]:
query = "What are the symptoms of asthma?"
retrieved_docs = vector_store.similarity_search(query, k=3)

print("Top Retrieved Documents:")
for doc in retrieved_docs:
    # print the source
    print(f"Source: {doc.metadata['source']}")
    # print the first 300 characters of the page content
    print(doc.page_content[:300])  # Print a snippet
    print("\n")


Top Retrieved Documents:
Source: ../docs\3.pdf
Home  Health  Conditions and Diseases
Signs of Respiratory
Distress
Signs of Respiratory Distress
Learning the signs of respiratory
distress
People having trouble breathing often show signs that they are having to work
harder to breathe or are not getting enough oxygen, indicating respiratory
dist


Source: ../docs\2.pdf
Global Regions
Donate
Home / Health topics / Chronic respiratory diseases
CreditsChronic respiratory diseases
Chronic respiratory diseases (CRDs) a ect the airways and other structures of the lungs. Some of the most
common are chronic obstructive pulmonary disease (COPD), asthma, occupational lung d


Source: ../docs\4.pdf
For Everyone 
Situation Summary
About Respiratory Illnesses
Risk Factors
Prevention
Treatment
Data and Trends
Resources to Prepare for Flu, COVID-19, and RSV
Respiratory Virus Guidance
Health Care Providers 
Best Practices for Patient Care
Talking with Patients
  Sign up for Email Updates  
ESPA




In [30]:
%pip install -q  langchain_openai ctransformers sacremoses


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [31]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# Load the model and tokenizer explicitly
model_name = "Qwen/Qwen2-72B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Set up Hugging Face pipeline for text generation
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

c:\Users\weszi\AppData\Local\Programs\Python\Python38\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\weszi\.cache\huggingface\hub\models--Qwen--Qwen2-72B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


KeyboardInterrupt: 

In [ ]:
# Provide a specific prompt
prompt = "What are the symptoms of asthma?"

# Generate text with proper truncation and max length
response = generator(prompt,
                     max_length=200,  # Increase max length
                     num_return_sequences=1,
                     truncation=True,  # Ensure truncation
                     do_sample=True,   # Enable sampling for variety
                     top_p=0.9,        # Control diversity
                     temperature=0.7)  # Control randomness

print("Generated Response:", response[0]['generated_text'])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Response: What are the symptoms of asthma?

Many people experience a variety of symptoms that include:

High blood pressure

Irregular breathing

Shortness of breath

Fatigue

Fatigue

Low blood pressure

Fatigue

Low blood cholesterol

Low HDL

Low cholesterol

Low total cholesterol

Low LDL

Low HDL

Low LDL

Low triglycerides

Low HDL

Low total cholesterol

Low total cholesterol

Low total cholesterol

Low total cholesterol

Low total cholesterol

Low total cholesterol

Low total cholesterol

Low total cholesterol

Low total cholesterol

Low total cholesterol

Low total cholesterol

Low total cholesterol

Low total cholesterol

Low total cholesterol

Low total cholesterol

Low total cholesterol

Low total cholesterol

Low total cholesterol

Low total cholesterol

Low total cholesterol

Low total cholesterol

Low total


In [ ]:
# Ensure pad_token is set if not already
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use EOS token as padding if not set

# Extract the content of the top retrieved documents, limiting the total length
retrieved_content = " ".join([doc.page_content for doc in retrieved_docs])
# Limit the retrieved content to a specific number of characters
max_retrieved_content_length = 200  # Adjust as needed
retrieved_content = retrieved_content[:max_retrieved_content_length]

# Update the prompt by appending the retrieved content
contextual_prompt = f"Context: {retrieved_content}\n\nQuestion: {prompt}\nAnswer:"

# Tokenize the input, further reducing max_length to avoid exceeding model's limit
encoded_input = tokenizer.encode(contextual_prompt, truncation=True, max_length=250, padding="max_length", return_tensors="pt")  # Reduced max_length to 250

# Check the tokenized input length
input_length = len(encoded_input[0])

# Calculate remaining space for generated tokens
remaining_tokens = 1024 - input_length  # Model limit is 1024 tokens (adjust as needed)

# Ensure that the generation does not exceed the model's token limit
response = generator(contextual_prompt,
                     max_new_tokens=min(remaining_tokens, 150),  # Limit the number of generated tokens
                     num_return_sequences=1,
                     do_sample=True,   # Enable sampling for variety
                     top_p=0.9,        # Control diversity
                     temperature=0.7,  # Control randomness
                     pad_token_id=tokenizer.eos_token_id)  # Explicitly use eos_token_id for padding
# Display the generated response
print("Generated Response:", response[0]['generated_text'])

Tokenized Input Length: 250
Remaining Tokens for Generation: 774
Generated Response: Context: Home  Health  Conditions and Diseases
Signs of Respiratory
Distress
Signs of Respiratory Distress
Learning the signs of respiratory
distress
People having trouble breathing often show signs that the

Question: What are the symptoms of asthma?
Answer: The symptoms of asthma
occur when you have the

difficulty breathing, lack of oxygen, or have a cough. It's usually caused by

bad airways, but it can also be caused by a combination of

other factors.
A cough can be caused by a combination of coughing

and/or a cough that comes from a cough, and it can also be caused by

bad airways, such as breathing in, or coughing up.
The symptoms of asthma
occur when you have the

difficulty breathing, lack of oxygen, or have a cough. It's usually

caused by bad airways, but it can also be caused by

bad airways,
